# Quickstart


In [4]:
import matplotlib.pyplot as plt
import os
import sys

#-------------------------------------------------------
# Import de tracklib

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

from tracklib.core.GPSTime import GPSTime
from tracklib.io.GpxReader import GpxReader
from tracklib.core.Operator import Operator
from tracklib.core.Plot import Plot

import tracklib.algo.Analytics as algo
import tracklib.algo.Interpolation as interp
import tracklib.algo.Segmentation as segmentation

import tracklib.core.Grid as grid
import tracklib.core.TrackCollection as trackCollection
import tracklib.algo.Summarising as summ


In [6]:
# ---------------------------------------------------
# Lecture des donnees
# ---------------------------------------------------
GPSTime.setReadFormat("4Y-2M-2DT2h:2m:2sZ")

tracks = GpxReader.readFromGpx('data/activity_5807084803.gpx')
trace = tracks.getTrack(0)

# Transformation GEO coordinates to ENU
trace.toENUCoords()

# Display
trace.plot()


FileNotFoundError: [Errno 2] No such file or directory: 'data/activity_5807084803.gpx'